In [21]:
# IMPORTING LIBRARIES
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import cv2
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout,BatchNormalization
import numpy as np
import tensorflow as tf

# PRAJAKTA KAPRE   22103040
# KAGGLE USERNAME: Prajakta Kapre

***Previously, I used the conventional Convolution layer and pooling layer architecture dor training but the accuracy was coming around 65%

**In this code,I have used the concept of Transfer learning using Xception model.
Firsty the Xception model is loaded without the top (fully connected) layers and new
fully connected layers are added on top of the base model.
After the initial training, the base model layers are unfrezzed for fine-tuning, recompiled the model and continude training with a lower learning rate.** *
**********

In [22]:
# USING THE CONCEPT OF TRANSFER LEARNING
# Load the Xception model without the top (fully connected) layers

base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3))



In [23]:
# Add your own fully connected layers on top of the base model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(400, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [24]:
# Freeze the base model layers to prevent further training
for layer in base_model.layers:
    layer.trainable = False

In [25]:
# Compile the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [26]:
# Using the Image Data Generator to import the images from the dataset 
#and splitting into training and validation sets

train_datagen = ImageDataGenerator(rescale = 1./255,         #min-max normalization
                                    #shear_range = 0.2,
                                    #zoom_range = 0.2,
                                    #horizontal_flip = True,
                                   validation_split=0.2)


train_generator = train_datagen.flow_from_directory('/kaggle/input/crack-detection-image-classification-2023/train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'binary',
                                                 subset='training')

validation_generator = train_datagen.flow_from_directory('/kaggle/input/crack-detection-image-classification-2023/train',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'binary',
                                            subset='validation')

Found 11975 images belonging to 2 classes.
Found 2993 images belonging to 2 classes.


In [27]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

Epoch 1/20
374/374 [==============================] - 84s 217ms/step - loss: 0.4873 - accuracy: 0.7577 - val_loss: 0.4425 - val_accuracy: 0.7970
Epoch 2/20
374/374 [==============================] - 81s 215ms/step - loss: 0.4295 - accuracy: 0.7915 - val_loss: 0.4288 - val_accuracy: 0.7974
Epoch 3/20
374/374 [==============================] - 85s 228ms/step - loss: 0.4037 - accuracy: 0.8101 - val_loss: 0.4199 - val_accuracy: 0.8007
Epoch 4/20
374/374 [==============================] - 82s 218ms/step - loss: 0.3788 - accuracy: 0.8225 - val_loss: 0.4008 - val_accuracy: 0.8085
Epoch 5/20
374/374 [==============================] - 85s 226ms/step - loss: 0.3633 - accuracy: 0.8304 - val_loss: 0.4033 - val_accuracy: 0.8142
Epoch 6/20
374/374 [==============================] - 80s 214ms/step - loss: 0.3454 - accuracy: 0.8433 - val_loss: 0.4548 - val_accuracy: 0.7765
Epoch 7/20
374/374 [==============================] - 80s 215ms/step - loss: 0.3262 - accuracy: 0.8512 - val_loss: 0.4258 - val_ac

In [28]:
# Unfreeze the base model layers for fine-tuning

for layer in base_model.layers:
    layer.trainable = True


In [29]:
# Compile the model again for fine-tuning

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [30]:
# Fine-tune the model

history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)


Epoch 1/20
374/374 [==============================] - 275s 672ms/step - loss: 0.4999 - accuracy: 0.7641 - val_loss: 0.4097 - val_accuracy: 0.8269
Epoch 2/20
374/374 [==============================] - 251s 670ms/step - loss: 0.3232 - accuracy: 0.8649 - val_loss: 0.6598 - val_accuracy: 0.7406
Epoch 3/20
374/374 [==============================] - 256s 683ms/step - loss: 0.2713 - accuracy: 0.8878 - val_loss: 0.3488 - val_accuracy: 0.8528
Epoch 4/20
374/374 [==============================] - 251s 671ms/step - loss: 0.2116 - accuracy: 0.9186 - val_loss: 0.3680 - val_accuracy: 0.8592
Epoch 5/20
374/374 [==============================] - 255s 682ms/step - loss: 0.1832 - accuracy: 0.9296 - val_loss: 0.4072 - val_accuracy: 0.8508
Epoch 6/20
374/374 [==============================] - 251s 671ms/step - loss: 0.1525 - accuracy: 0.9417 - val_loss: 0.4867 - val_accuracy: 0.8256
Epoch 7/20
374/374 [==============================] - 251s 671ms/step - loss: 0.1366 - accuracy: 0.9472 - val_loss: 0.4350 -

In [31]:
# FINDING VALIDATION ACCURACY

model.evaluate(validation_generator)

94/94 [==============================] - 16s 171ms/step - loss: 0.6318 - accuracy: 0.8647


[0.6317837834358215, 0.8646842837333679]

In [33]:
# GETTING OUTPUTS FROM CREATED MODEL

test_path = '/kaggle/input/crack-detection-image-classification-2023/test'
test_filenames = os.listdir(test_path)
             
              
test_predictions = np.array([])
test_labels=[]

for img in test_filenames:
    image_path = os.path.join(test_path,img)
    image_path
    image = tf.keras.utils.load_img(image_path, target_size = (256,256))
    image = tf.keras.utils.img_to_array(image)
    image = np.array([image])
    image = image/255
    predictions = np.squeeze((model.predict(image) >= 0.5))
    test_predictions = np.append(test_predictions, predictions)
    if predictions==1:
        test_labels.append("uncracked")
    elif predictions==0:
        test_labels.append("cracked")

print(test_predictions)

1/1 [==============================] - 0s 25ms/step
[1. 0. 1. ... 0. 0. 0.]


In [36]:
test_predictions.shape

(2000,)

In [37]:
# FINAL OUTPUT CREATION FOR KAGGLE SUBMISSION

filename=os.listdir('/kaggle/input/crack-detection-image-classification-2023/test')

data = pd.DataFrame({'filename':filename,'class':test_predictions})
data['class']=data['class'].apply(str)
data
data['class']=data['class'].str.replace('1.0','uncracked')
data['class']=data['class'].str.replace('0.0','cracked')
data.to_csv('/kaggle/working/output9_assg2.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
